In [2]:
import pandas as pd

import requests
from bs4 import BeautifulSoup
import json
from urllib.request import urlopen
from tqdm import tqdm_notebook
import unidecode


In [2]:
df_debut = pd.read_csv("../data/debut_album_1118-utf8.csv")

In [3]:
def get_genius(data):
    
    df_articles = pd.DataFrame(columns = ['artist', 'album', 'genius_frequency'])
    
    for i in tqdm_notebook(range(len(list(df_debut['artist'])))):

        artist = df_debut['artist'][i]
        album = df_debut['album'][i]
        
        artist_ = artist.replace(" ", "+")
        album_ = album.replace(" ", "+")
        
        page_num = 1
        articles = 0

        while True:

            url_ = 'https://genius.com/api/search/article?page=' + str(page_num) + '&q=' + artist_ + "+" + album_
            response = requests.get(url_).json()

            articles += len(response['response']['sections'][0]['hits'])

            page_num += 1

            if response['response']['next_page'] == None :
                break

        data = {
            'artist': artist,
            'album' : album,
            'genius_frequency' : articles
        }

#         print(data)

        df_articles.loc[len(df_articles)] = data
    
    return df_articles

In [4]:
genius_articles = get_genius(df_debut)

HBox(children=(IntProgress(value=0, max=1951), HTML(value='')))

In [5]:
print(genius_articles.shape)
genius_articles.tail()

(1951, 3)


,artist,album,genius_frequency
1946,EDEN,vertigo,0
1947,Nipsey Hussle,Victory Lap,1
1948,Rich the Kid,The World Is Yours,1
1949,Myles Kennedy,Year of the Tiger,0
1950,Fickle Friends,You Are Someone Else,0


In [6]:
def decodeuni(str):
    return unidecode.unidecode(str)

In [8]:
genius_articles['artist'] = genius_articles['artist'].apply(decodeuni)
genius_articles['album'] = genius_articles['album'].apply(decodeuni)
genius_articles['artist'] = genius_articles['artist'].apply(lambda x : x.replace(",", ""))
genius_articles['album'] = genius_articles['album'].apply(lambda x : x.replace(",", ""))

In [28]:
genius_articles.loc[2, 'album'] = '9am to 5pm 5pm to Whenever'
genius_articles.loc[529, 'album'] = 'If You Love Me'
genius_articles.loc[1144, 'album'] = 'Gover Tela Body Language'
genius_articles.loc[1413, 'album'] = 'Fresh Filter Volumn 1'

In [32]:
genius_articles.shape

(1951, 3)

In [33]:
genius_articles.to_csv("../data/buzz_genius.csv", index=False, encoding='UTF-8')

In [35]:
df = pd.read_csv("../data/buzz_genius-utf8.csv")

In [37]:
df.shape

(1951, 3)

### Upload to MySQL database

In [38]:
import sqlalchemy, pickle
from sqlalchemy import create_engine

In [39]:
pw = pickle.load(open("mysql_pw.pickle", "rb"))

In [40]:
engine = sqlalchemy.create_engine("mysql+mysqldb://root:" + pw + "@52.78.44.120/project_rookie")

In [41]:
df.to_sql(name="buzz_genius", con=engine, if_exists='replace')